# Label / Connected Components

Use the `label` function from `elf.parallel` to label a volumetric segmentation by applying connected components.

In [ ]:
%gui qt5 

# import napari for data visualisation
import napari
import numpy as np

# import the relevant elf functionality
from elf.io import open_file
from elf.parallel.label import label
from elf.evaluation import rand_index

# use skimage.label to verify the elf implementation
from skimage.measure import label as label_reference

In [ ]:
# read the data
# you can download the example data from here:
# https://hcicloud.iwr.uni-heidelberg.de/index.php/s/6LuE7nxBN3EFRtL
data_path = '/home/pape/Work/data/isbi/isbi_train_volume.h5'  # adjust this path
with open_file(data_path, 'r') as f:
    # load the raw data
    raw = f['raw'][:]
    # load the segmentation
    seg = f['labels/gt_segmentation'][:]

In [ ]:
def merge_ids(seg, n_merges=50):
    uniques = np.unique(seg)
    merge_ids = np.random.choice(uniques,
                                 size=(n_merges, 2),
                                 replace=False)
    for ida, idb in merge_ids:
        seg[seg == idb] = ida
    return seg

In [ ]:
# merge some of the segment ids,
# so that connected components has an effect
seg = merge_ids(seg)

In [ ]:
# apply connected components to label the volume
blocks = (10, 100, 100)  # block size used for parallelisation
# allocate the output data (label cannot be applied inplace)
labeled = np.zeros_like(seg)
labeled = label(seg, labeled,
                block_shape=blocks, verbose=True)

In [ ]:
# compute label with the skimage reference implementation
labeled_ref = label_reference(seg)
# check that the results agree via the rand index
_, ri = rand_index(labeled, labeled_ref)
print("The rand index with the reference implementation is", ri)
print("(1. means complete agreement.)")

In [ ]:
# visualize the results
viewer = napari.Viewer()
viewer.add_image(raw, name='raw')
viewer.add_labels(seg, name='segmentation')
viewer.add_labels(labeled, name='labeled')